### Team1 - Neural Fusion


### Introduction 

Objective:

To solve a series of challenges, each leading closer to uncovering the final treasure. 

Along the way, we are to employ text processing techniques to decode hidden clues embedded within texts,to decode clues, uncover hidden connections, and collaborate with others to reach the ultimate treasure! 

Clues for Group 1
1. Clue 1a (Easier): From lines and shapes to scenes so bright, I give digital creations the illusion of light.
2. Clue 2a (Harder): Where polygons dance and shaders weave, my artistry helps the virtual world breathe.
3. Clue 1 b(Easier): Tiny squares, a canvas wide, building images side by side.
4. Clue 2b (Harder): RGB spells my secret code, the building blocks where colors explode.



### Install the necessary libraries - Setting Up 

#ensuring we have all the necessary libraries installed and importing them into our Python environment

#The libraries include : NLTK, Pandas, Scikit-learn, Gensim, and Spacy. Additionally, we provide an optional section for advanced exploration with Transformers, which requires installation and importation of the Transformers library.



In [1]:

!pip install torch
!pip install --upgrade torch transformers
!pip install Matplotlib
!pip install seaborn


     -------------------------------------- 159.7/159.7 MB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 9.0/9.0 MB 36.0 MB/s eta 0:00:00
     -------------------------------------- 228.5/228.5 MB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 20.1 MB/s eta 0:00:00
     ---------------------------------------- 3.5/3.5 MB 37.3 MB/s eta 0:00:00
     ------------------------------------- 286.4/286.4 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: tbb
    Found existing installation: TBB 0.2


ERROR: Cannot uninstall 'TBB'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [2]:
# Import libraries
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from nltk.corpus import stopwords
import nltk

import numpy as np
import pandas as pd
import math
import os
#import system

import matplotlib.pyplot as plt #data viz
import seaborn as sns # data viz
#supress warnings
import warnings
warnings.filterwarnings('ignore')

nltk.download("stopwords")
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec  # For Word2Vec embeddings
import re  # For regular expressions

# Optional advanced exploration with Transformers
from transformers import pipeline


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fisay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Quest Begins – The Initial Clue
*Clue 1a (Easier): From lines and shapes to scenes so bright, I give digital creations the illusion of light.

In [3]:
# Load the 20 newsgroups dataset for 'sci.med' and 'sci.space' categories

categories = ["sci.med", "sci.space"]
newsgroups_train = fetch_20newsgroups(subset="train", categories=categories)
newsgroups_test = fetch_20newsgroups(subset="test", categories=categories)

# Create a dataframe for 'sci.med' and 'sci.space' categories
df = pd.DataFrame(data=newsgroups_train.data, columns=["text"])
df["target"] = newsgroups_train.target
df["category"] = df["target"].map(lambda x: categories[x])

# Display the number of rows and columns
df.shape

(1187, 3)

In [4]:
# Display the first few rows to test
df.head()

,text,target,category
0,"From: flb@flb.optiplan.fi (""F.Baube[tm]"")\nSub...",1,sci.space
1,From: geb@cs.pitt.edu (Gordon Banks)\nSubject:...,0,sci.med
2,From: ab961@Freenet.carleton.ca (Robert Alliso...,0,sci.med
3,From: rind@enterprise.bih.harvard.edu (David R...,0,sci.med
4,From: nsmca@aurora.alaska.edu\nSubject: Space ...,1,sci.space


In [5]:
df.tail()

,text,target,category
1182,From: clarke@acme.ucf.edu (Thomas Clarke)\nSub...,1,sci.space
1183,From: bruce@Data-IO.COM (Bruce Reynolds)\nSubj...,0,sci.med
1184,From: baalke@kelvin.jpl.nasa.gov (Ron Baalke)\...,1,sci.space
1185,From: 18084TM@msu.edu (Tom)\nSubject: Level 5?...,1,sci.space
1186,From: steveo@world.std.com (Steven W Orr)\nSub...,0,sci.med


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187 entries, 0 to 1186
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      1187 non-null   object
 1   target    1187 non-null   int64 
 2   category  1187 non-null   object
dtypes: int64(1), object(2)
memory usage: 27.9+ KB


In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
target,1187.0,0.499579,0.500211,0.0,0.0,0.0,1.0,1.0


# Keyword Quest
# Starting with TF-IDF:


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  # Import cosine_similarity
from nltk.corpus import stopwords

def extract_keywords(text_data, clues, top_n=10):
    # Preprocess the text data
    stop_words = set(stopwords.words('english'))
    preprocessed_data = [' '.join([word for word in text.lower().split() if word not in stop_words]) for text in text_data]

    # Create a TF-IDF vectorizer
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(preprocessed_data)

    # Get the feature names (keywords)
    feature_names = vectorizer.get_feature_names_out()

    # Preprocess the clues
    preprocessed_clues = [' '.join([word for word in clue.lower().split() if word not in stop_words]) for clue in clues]

    # Transform the clues using the same vectorizer
    clue_vectors = vectorizer.transform(preprocessed_clues)

    # Calculate cosine similarity between clues and text data
    similarity_scores = cosine_similarity(clue_vectors, tfidf_matrix)

    # Find the top N most similar keywords for each clue
    top_keywords = []
    for i in range(len(clues)):
        clue_scores = similarity_scores[i]
        top_indices = clue_scores.argsort()[-top_n:][::-1]
        top_keywords.append([feature_names[index] for index in top_indices])

    return top_keywords

clues = [
    "From lines and shapes to scenes so bright, I give digital creations the illusion of light.",
    "Where polygons dance and shaders weave, my artistry helps the virtual world breathe."
]

text_data = [
    "Digital art encompasses a range of artistic work and practices that use digital technology as an essential part of the creative and/or presentation process.",
    "It involves creating artwork digitally, either by computer software or other digital tools.",
    "The resulting artwork can be printed, displayed on a screen, or projected onto a surface."
]

# Extract relevant keywords based on the clues
relevant_keywords = extract_keywords(text_data, clues)

# Print the relevant keywords for each clue
for i, clue in enumerate(clues):
    print(f"Clue: {clue}")
    print("Relevant keywords:", relevant_keywords[i])
    print()

Clue: From lines and shapes to scenes so bright, I give digital creations the illusion of light.
Relevant keywords: ['and', 'art', 'artistic']

Clue: Where polygons dance and shaders weave, my artistry helps the virtual world breathe.
Relevant keywords: ['artistic', 'art', 'and']



# Clues

In [9]:
import re

# Clues
scimed_Clue_1a = (
    "From lines and shapes to scenes so bright, I give digital creations the illusion of light."
)
scimed_Clue_2a = (
    "Where polygons dance and shaders weave, my artistry helps the virtual world breathe."
)
scimed_Clue_1b = (
    "Tiny squares, a canvas wide, building images side by side."
)
scimed_Clue_2b = (
    "RGB spells my secret code, the building blocks where colors explode."
)


# Sci Space topic
scispace_Clue_1a = "From lines and shapes to scenes so bright, I give digital creations the illusion of light."
scispace_Clue_2a = "Where polygons dance and shaders weave, my artistry helps the virtual world breathe."
scispace_Clue_1b = (
    "Tiny squares, a canvas wide, building images side by side."
)
scispace_Clue_2b = "RGB spells my secret code, the building blocks where colors explode."

scimed_clues_list = [
    scimed_Clue_1a,
    scimed_Clue_2a,
    scimed_Clue_1b,
    scimed_Clue_2b,
    
]
scispace_clues_list = [
    scispace_Clue_1a,
    scispace_Clue_2a,
    scispace_Clue_1b,
    scispace_Clue_2b,
]

# clean the clues list text
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    return text

scimed_clues_list = [clean_text(clue) for clue in scimed_clues_list]
scispace_clues_list = [clean_text(clue) for clue in scispace_clues_list]

print(f"Sci Med Clues: {scimed_clues_list}")
print("----------------")
print(f"Sci Space Clues: {scispace_clues_list}")


Sci Med Clues: ['from lines and shapes to scenes so bright i give digital creations the illusion of light', 'where polygons dance and shaders weave my artistry helps the virtual world breathe', 'tiny squares a canvas wide building images side by side', 'rgb spells my secret code the building blocks where colors explode']
----------------
Sci Space Clues: ['from lines and shapes to scenes so bright i give digital creations the illusion of light', 'where polygons dance and shaders weave my artistry helps the virtual world breathe', 'tiny squares a canvas wide building images side by side', 'rgb spells my secret code the building blocks where colors explode']


### Word embeddings like Word2Vec or GloVe help us understand how words relate to each other.

# Topkey & Related  words

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import re


# Clues
scimed_Clue_1a = (
    "From lines and shapes to scenes so bright, I give digital creations the illusion of light."
)
scimed_Clue_2a = (
    "Where polygons dance and shaders weave, my artistry helps the virtual world breathe."
)
scimed_Clue_1b = (
    "Tiny squares, a canvas wide, building images side by side."
)
scimed_Clue_2b = (
    "RGB spells my secret code, the building blocks where colors explode."
)


# Sci Space topic
scispace_Clue_1a = "From lines and shapes to scenes so bright, I give digital creations the illusion of light."
scispace_Clue_2a = "Where polygons dance and shaders weave, my artistry helps the virtual world breathe."
scispace_Clue_1b = (
    "Tiny squares, a canvas wide, building images side by side."
)
scispace_Clue_2b = "RGB spells my secret code, the building blocks where colors explode."

scimed_clues_list = [
    scimed_Clue_1a,
    scimed_Clue_2a,
    scimed_Clue_1b,
    scimed_Clue_2b,
    
]
scispace_clues_list = [
    scispace_Clue_1a,
    scispace_Clue_2a,
    scispace_Clue_1b,
    scispace_Clue_2b,
]

# clean the clues list text
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    return text

scimed_clues_list = [clean_text(clue) for clue in scimed_clues_list]
scispace_clues_list = [clean_text(clue) for clue in scispace_clues_list]

def tfidf_extract_keywords(clues_list):
    # Create a TF-IDF Vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words="english")

    # Fit the vectorizer to the clues
    tfidf_matrix = tfidf_vectorizer.fit_transform(clues_list)

    # Get the feature names of `tfidf_vectorizer`
    feature_names = tfidf_vectorizer.get_feature_names_out()

    # Create a DataFrame of the `tfidf_matrix`
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

    # Get the top feature from each clue
    top_keywords = tfidf_df.idxmax(axis=1).values
    return top_keywords

scimed_related_keywords_tfidf = tfidf_extract_keywords(scimed_clues_list)
scispace_related_keywords_tfidf = tfidf_extract_keywords(scispace_clues_list)

print(f"Top keywords for 'sci.med' clues: {scimed_related_keywords_tfidf}")
print("----------------")
print(f"Top keywords for 'sci.space' clues: {scispace_related_keywords_tfidf}")


Top keywords for 'sci.med' clues: ['bright' 'artistry' 'canvas' 'blocks']
----------------
Top keywords for 'sci.space' clues: ['bright' 'artistry' 'canvas' 'blocks']


# Glove embeddings 

In [18]:
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Load pre-trained GloVe embeddings
glove_file = "20newsgroups.txt"
glove_embeddings = KeyedVectors.load_word2vec_format(
    glove_file, binary=False, no_header=True
)


def tfidf_extract_keywords(clues_list):
    # Create a TF-IDF Vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words="english")

    # Fit the vectorizer to the clues
    tfidf_matrix = tfidf_vectorizer.fit_transform(clues_list)

    # Get the feature names of `tfidf_vectorizer`
    feature_names = tfidf_vectorizer.get_feature_names_out()

    # Create a DataFrame of the `tfidf_matrix`
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

    # Get the top feature from each clue
    top_keywords = tfidf_df.idxmax(axis=1).values
    return top_keywords

# Make sure to define scimed_clues_list and scispace_clues_list with correct clues before calling tfidf_extract_keywords function
scimed_related_keywords_glove = tfidf_extract_keywords(scimed_clues_list)
scispace_related_keywords_glove = tfidf_extract_keywords(scispace_clues_list)

print(f"Related keywords for 'sci.med' clues (GloVe): {scimed_related_keywords_glove}")
print("----------------")
print(f"Related keywords for 'sci.space' clues (GloVe): {scispace_related_keywords_glove}")



Related keywords for 'sci.med' clues (GloVe): ['bright' 'artistry' 'canvas' 'blocks']
----------------
Related keywords for 'sci.space' clues (GloVe): ['bright' 'artistry' 'canvas' 'blocks']


## reveal new concepts, or hint at hidden patterns within the data.
## Word2Vec 

In [19]:
from gensim.models import Word2Vec

# Function to get related keywords using Word2Vec
def get_related_keywords(clues_list, keywords, n=5):
    # Tokenize the clues
    tokenized_clues = [clue.split() for clue in clues_list]

    # Create a Word2Vec model
    word2vec = Word2Vec(tokenized_clues, vector_size=100, window=5, min_count=1, sg=1)

    related_keywords = []
    for keyword in keywords:
        try:
            # Get the most similar keywords for each input keyword
            related = word2vec.wv.most_similar(positive=[keyword], topn=n)
            related_keywords.extend([w[0] for w in related])
        except KeyError:
            # Skip keywords not in the vocabulary
            pass

    return related_keywords


# Define scimed_keywords (example)
scimed_keywords = ['artistry', 'blocks', 'breathe', 'bright', 'building']

# Get related keywords for 'sci.med' clues
scimed_related_keywords = get_related_keywords(scimed_clues_list, scimed_keywords)

print(f"Related keywords for 'sci.med' clues: {scimed_related_keywords}")


Related keywords for 'sci.med' clues: ['and', 'colors', 'spells', 'code', 'scenes', 'dance', 'digital', 'give', 'breathe', 'shapes', 'digital', 'i', 'polygons', 'side', 'blocks', 'explode', 'images', 'my', 'a', 'virtual', 'creations', 'by', 'side', 'where', 'artistry']


# Semantic Safari

## Exploring the World of Meaning: Word embeddings like Word2Vec or GloVe help us understand how words relate to each other.


In [20]:
from gensim.models import Word2Vec

# Function to calculate similarity between keywords and clues
def calculate_similarity(keywords, clues_list):
    # Tokenize the clues
    tokenized_clues = [clue.split() for clue in clues_list]

    # Create a Word2Vec model
    word2vec = Word2Vec(tokenized_clues, vector_size=100, window=5, min_count=1, sg=1)

    similarities = []
    for keyword in keywords:
        try:
            # Get the most similar clues for each keyword
            similar_clues = word2vec.wv.most_similar(positive=[keyword], topn=1)
            similarities.append((keyword, similar_clues[0][0], similar_clues[0][1]))
        except KeyError:
            # Skip keywords not in the vocabulary
            pass

    return similarities

# Define scispace_keywords (example)
scispace_keywords = ['space', 'galaxy', 'astronomy', 'cosmos', 'planet']

# Calculate similarities between 'sci.med' keywords and 'sci.space' clues, and vice versa
scimed_similarities = calculate_similarity(scimed_keywords, scispace_clues_list)
scispace_similarities = calculate_similarity(scispace_keywords, scimed_clues_list)

print(
    f"Similarities between 'sci.med' keywords and 'sci.space' clues: {scimed_similarities}"
)
print("----------------")
print(
    f"Similarities between 'sci.space' keywords and 'sci.med' clues: {scispace_similarities}"
)


Similarities between 'sci.med' keywords and 'sci.space' clues: [('artistry', 'and', 0.2120572030544281), ('blocks', 'dance', 0.2164992243051529), ('breathe', 'digital', 0.3488596975803375), ('bright', 'explode', 0.1961306929588318), ('building', 'creations', 0.16702575981616974)]
----------------
Similarities between 'sci.space' keywords and 'sci.med' clues: []




## Trying to Calculate similarities between our keywords and texts in other categories. 


In [ ]:
# Calculate the average similarity score for 'sci.med' and 'sci.space' clues
scimed_similarity_score = np.mean([score for _, _, score in scimed_similarities])
scispace_similarity_score = np.mean([score for _, _, score in scispace_similarities])

print(f"Average similarity score for 'sci.med' clues: {scimed_similarity_score}")
print(f"Average similarity score for 'sci.space' clues: {scispace_similarity_score}")



# why is the confidence score low? ***

# Advanced Exploration: Transformers ** (Optional)

Transformer-based models provide even more nuanced semantic understanding. 

In [21]:
from transformers import pipeline


# Function to perform question answering using a pre-trained model
def answer_question(question, context):
    answerer = pipeline("question-answering")
    answer = answerer({"question": question, "context": context})
    return answer


# Function to perform text classification using a pre-trained model
def classify_text(text):
    classifier = pipeline("text-classification")
    result = classifier(text)
    return result


# Example usage for question answering
scimed_question = "What influences mood in the brain?"
scimed_context = " ".join(scimed_clues_list)
scimed_answer = answer_question(scimed_question, scimed_context)
print(f"Question: {scimed_question}")
print(f"Answer: {scimed_answer['answer']}")
print(f"Score: {scimed_answer['score']}")
print("----------------")

scispace_question = "What is the goal of space exploration?"
scispace_context = " ".join(scispace_clues_list)
scispace_answer = answer_question(scispace_question, scispace_context)
print(f"Question: {scispace_question}")
print(f"Answer: {scispace_answer['answer']}")
print(f"Score: {scispace_answer['score']}")
print("----------------")

# Example usage for text classification
scimed_text = "This text seems to be about neurotransmitters and their effects on mood."
scimed_classification = classify_text(scimed_text)
print(f"Text: {scimed_text}")
print(f"Classification: {scimed_classification[0]['label']}")
print(f"Score: {scimed_classification[0]['score']}")
print("----------------")

scispace_text = (
    "This text seems to be about space exploration and technological advancements."
)
scispace_classification = classify_text(scispace_text)
print(f"Text: {scispace_text}")
print(f"Classification: {scispace_classification[0]['label']}")
print(f"Score: {scispace_classification[0]['score']}")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Question: What influences mood in the brain?
Answer: light
Score: 0.06852194666862488
----------------


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Question: What is the goal of space exploration?
Answer: helps the virtual world breathe tiny squares
Score: 0.22464647889137268
----------------


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Text: This text seems to be about neurotransmitters and their effects on mood.
Classification: NEGATIVE
Score: 0.9496363401412964
----------------
Text: This text seems to be about space exploration and technological advancements.
Classification: POSITIVE
Score: 0.9836159348487854


## Take 2 

In [29]:
# Function to perform question answering using a fine-tuned model
def answer_question(question, context):
    device = 0 if torch.cuda.is_available() else -1  # Check if GPU is available
    answerer = pipeline("question-answering", model="deepset/roberta-base-squad2", device=device)
    answer = answerer({"question": question, "context": context})
    return answer

# Function to perform text classification using a fine-tuned model
def classify_text(text):
    device = 0 if torch.cuda.is_available() else -1  # Check if GPU is available
    classifier = pipeline("sentiment-analysis", model="textattack/bert-base-uncased-SST-2", device=device)
    result = classifier(text)
    return result

# Updated context for the questions
scimed_context = " ".join(scimed_clues_list)
scispace_context = " ".join(scispace_clues_list)

# Example usage for question answering
scimed_answer = answer_question("What influences mood in the brain?", scimed_context)
print("Question: What influences mood in the brain?")
print(f"Answer: {scimed_answer['answer']}")
print(f"Score: {scimed_answer['score']}")
print("----------------")

scispace_answer = answer_question("What is the goal of space exploration?", scispace_context)
print("Question: What is the goal of space exploration?")
print(f"Answer: {scispace_answer['answer']}")
print(f"Score: {scispace_answer['score']}")
print("----------------")

# Example usage for text classification
scimed_classification = classify_text("This text seems to be about neurotransmitters and their effects on mood.")
print("Text: This text seems to be about neurotransmitters and their effects on mood.")
print(f"Classification: {'Positive' if scimed_classification[0]['label'] == 'LABEL_1' else 'Negative'}")
print(f"Score: {scimed_classification[0]['score']}")
print("----------------")

scispace_classification = classify_text("This text seems to be about space exploration and technological advancements.")
print("Text: This text seems to be about space exploration and technological advancements.")
print(f"Classification: {'Positive' if scispace_classification[0]['label'] == 'LABEL_1' else 'Negative'}")
print(f"Score: {scispace_classification[0]['score']}")


Question: What influences mood in the brain?
Answer: lines and shapes
Score: 2.6440488909429405e-06
----------------
Question: What is the goal of space exploration?
Answer: building blocks where colors explode
Score: 1.211475506579518e-07
----------------
Text: This text seems to be about neurotransmitters and their effects on mood.
Classification: Negative
Score: 0.8315242528915405
----------------
Text: This text seems to be about space exploration and technological advancements.
Classification: Positive
Score: 0.9824037551879883


# take 3 

In [35]:
from transformers import pipeline
import warnings

# Suppress default model warnings
warnings.filterwarnings("ignore", message=".*Using a pipeline.*")  # Silence warnings


def get_question_answering_model(task_name="squad"):
  """
  Fetches a question answering model based on the specified task.

  Args:
      task_name (str, optional): The task for which the model is optimized.
          Defaults to "squad".

  Returns:
      pipeline: A pipeline object loaded with the chosen question answering model.
  """
  # Consider using model selection libraries or exploring Hugging Face Hub for task-specific models
  if task_name == "squad":
    model_name = "distilbert/distilbert-base-cased-distilled-squad"
  else:
    # Add logic for choosing other question answering models based on task_name
    raise ValueError(f"Unsupported task name: {task_name}")
  return pipeline("question-answering", model=model_name)


def get_text_classification_model(task_name="sentiment-analysis"):
  """
  Fetches a text classification model based on the specified task.

  Args:
      task_name (str, optional): The task for which the model is optimized.
          Defaults to "sentiment-analysis".

  Returns:
      pipeline: A pipeline object loaded with the chosen text classification model.
  """
  # Consider using model selection libraries or exploring Hugging Face Hub for task-specific models
  if task_name == "sentiment-analysis":
    model_name = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
  else:
    # Add logic for choosing other text classification models based on task_name
    raise ValueError(f"Unsupported task name: {task_name}")
  return pipeline("text-classification", model=model_name)


# Function to perform question answering using a pre-trained model
def answer_question(question, context, model=None):
  """
  Answers a question using a pre-trained question answering model.

  Args:
      question (str): The question to be answered.
      context (str): The context in which to find the answer.
      model (pipeline, optional): A pre-loaded question answering pipeline object.
          If not provided, a default model will be fetched based on the task name.

  Returns:
      dict: A dictionary containing the answer and its score.
  """
  if model is None:
    question_answering_model = get_question_answering_model()
  else:
    question_answering_model = model
  answer = question_answering_model({"question": question, "context": context})
  return answer


# Function to perform text classification using a pre-trained model
def classify_text(text, model=None):
  """
  Classifies text using a pre-trained text classification model.

  Args:
      text (str): The text to be classified.
      model (pipeline, optional): A pre-loaded text classification pipeline object.
          If not provided, a default model will be fetched based on the task name.

  Returns:
      list: A list containing the classification label and its score.
  """
  if model is None:
    text_classification_model = get_text_classification_model()
  else:
    text_classification_model = model
  result = text_classification_model(text)
  return result


# Example usage
scimed_question = "What influences mood in the brain?"
# Replace with your actual list of clues for SciMed context
scimed_clues_list = ["Clue 1 about neurotransmitters", "Clue 2 about brain chemicals"]
scimed_context = " ".join(scimed_clues_list)

scispace_question = "What is the goal of space exploration?"
# Replace with your actual list of clues for SciSpace context
scispace_clues_list = ["Clue 1 about space exploration", "Clue 2 about space discoveries"]
scispace_context = " ".join(scispace_clues_list)

scimed_answer = answer_question(scimed_question, scimed_context)
print(f"Question: {scimed_question}")
print(f"Answer: {scimed_answer['answer']}")
print(f"Score: {scimed_answer['score']}")
print("----------------")

scispace_answer = answer_question(scispace_question, scispace_context)
# Print regardless of score (change 0.0 to a desired threshold if needed)
print(f"Question: {scispace_question}")
print(f"Answer: {scispace_answer['answer']}")
print(f"Score: {scispace_answer['score']}")
print("----------------")



Question: What influences mood in the brain?
Answer: brain chemicals
Score: 0.4653284251689911
----------------
Question: What is the goal of space exploration?
Answer: space discoveries
Score: 0.6120656132698059
----------------


 # Trying to identify the Pattern 


In [31]:
import re


##  Function to detect patterns in clues using regular expressions
def detect_patterns(clues_list):
    patterns = []
    for clue in clues_list:
        # Pattern for chemical names (e.g., serotonin, tryptophan)
        chemical_pattern = r"\b[A-Za-z]+\b"
        chemicals = re.findall(chemical_pattern, clue)

        # Pattern for scientific terms (e.g., neurotransmitter, microbiome)
        science_term_pattern = r"\b[A-Za-z]+\b"
        science_terms = re.findall(science_term_pattern, clue)

        # Pattern for space-related terms (e.g., rockets, gravity, satellites)
        space_term_pattern = r"\b[A-Za-z]+\b"
        space_terms = re.findall(space_term_pattern, clue)

        # Pattern for celestial bodies or phenomena (e.g., stars, cosmos)
        celestial_pattern = r"\b[A-Za-z]+\b"
        celestial_terms = re.findall(celestial_pattern, clue)

        if chemicals or science_terms or space_terms or celestial_terms:
            patterns.append(
                {
                    "clue": clue,
                    "chemicals": chemicals,
                    "science_terms": science_terms,
                    "space_terms": space_terms,
                    "celestial_terms": celestial_terms,
                }
            )

    return patterns


# Detect patterns in 'sci.med' and 'sci.space' clues
scimed_patterns = detect_patterns(scimed_clues_list)
scispace_patterns = detect_patterns(scispace_clues_list)

print("Detected patterns in 'sci.med' clues:")
for pattern in scimed_patterns:
    print(f"Clue: {pattern['clue']}")
    if pattern["chemicals"]:
        print(f"Chemicals: {pattern['chemicals']}")
    if pattern["science_terms"]:
        print(f"Science Terms: {pattern['science_terms']}")
    print("----------------")

print("Detected patterns in 'sci.space' clues:")
for pattern in scispace_patterns:
    print(f"Clue: {pattern['clue']}")
    if pattern["space_terms"]:
        print(f"Space Terms: {pattern['space_terms']}")
    if pattern["celestial_terms"]:
        print(f"Celestial Terms: {pattern['celestial_terms']}")
    print("----------------")

Detected patterns in 'sci.med' clues:
Clue: from lines and shapes to scenes so bright i give digital creations the illusion of light
Chemicals: ['from', 'lines', 'and', 'shapes', 'to', 'scenes', 'so', 'bright', 'i', 'give', 'digital', 'creations', 'the', 'illusion', 'of', 'light']
Science Terms: ['from', 'lines', 'and', 'shapes', 'to', 'scenes', 'so', 'bright', 'i', 'give', 'digital', 'creations', 'the', 'illusion', 'of', 'light']
----------------
Clue: where polygons dance and shaders weave my artistry helps the virtual world breathe
Chemicals: ['where', 'polygons', 'dance', 'and', 'shaders', 'weave', 'my', 'artistry', 'helps', 'the', 'virtual', 'world', 'breathe']
Science Terms: ['where', 'polygons', 'dance', 'and', 'shaders', 'weave', 'my', 'artistry', 'helps', 'the', 'virtual', 'world', 'breathe']
----------------
Clue: tiny squares a canvas wide building images side by side
Chemicals: ['tiny', 'squares', 'a', 'canvas', 'wide', 'building', 'images', 'side', 'by', 'side']
Science Te


# Technical Report: Text Treasure Hunt - The Vectorization Adventure

1. Introduction

In this report, we embark on the journey of "The Vectorization Adventure," where we delve into the world of text processing and vectorization. Our mission is to decode clues, uncover hidden connections, and collaborate with other teams to reach the ultimate treasure. The adventure involves several stages, each requiring different techniques and approaches to progress towards the final goal.

2. Setting Up

We start by setting up our environment, ensuring we have all the necessary libraries installed and importing them into our Python environment. The libraries include NLTK, Pandas, Scikit-learn, Gensim, and Spacy. Additionally, we provide an optional section for advanced exploration with Transformers, which requires installation and importation of the Transformers library.

3.  Quest Begins - The Initial Clue

The first step is to decipher the initial clue provided by our instructor. We carefully analyze the clue to identify words or themes that stand out, which will guide us towards selecting the relevant topic category within the Newsgroup 20 dataset.

4. Keyword Quest

With our initial clue in hand, we proceed to extract keywords using TF-IDF (Term Frequency-Inverse Document Frequency). We define a function extract_keywords that applies TF-IDF to the selected texts, extracting relevant keywords that may hint at the next topic or text to explore. The extracted keywords help illuminate our path forward.

5. Semantic Safari

Next, we embark on a semantic analysis journey using word embeddings like Word2Vec or GloVe. We calculate similarities between our extracted keywords and texts in other categories, seeking unexpected connections that may lead us closer to the treasure. 
Advanced exploration with Transformers is also provided for a deeper semantic understanding.

6. Pattern Pursuit

In this stage, we search for unusual patterns within the texts using regular expressions. We provide examples of regular expressions to find potential codes or emails within the text data. By examining letter sequences, numbers, or other patterns, we may uncover hidden clues crucial for progressing in the adventure.

7. Collaboration and Convergence

Teamwork is essential for success in this adventure. We discuss effective communication strategies for sharing findings and combining insights with other teams. Collaboration is key to solving the ultimate puzzle and locating the treasure by converging all the gathered clues.

8. Reflection and Report

Finally, we reflect on our journey, documenting the methods, techniques, and insights gained at each stage. We provide detailed explanations of the code snippets used and their significance in progressing through the adventure. We discuss the most helpful text processing techniques, the empowering nature of vectorization, surprising discoveries made, and potential real-world applications of our skills.

In conclusion, "The Vectorization Adventure" is a challenging yet rewarding journey through the realm of text processing and vectorization. By carefully analyzing clues, extracting keywords, exploring semantic meanings, and uncovering patterns, we inch closer to the ultimate treasure. Through collaboration and reflection, we not only solve the adventure but also gain valuable insights and skills applicable in various real-world scenarios.



### Reflection

Which text processing techniques were most helpful and why?
- TF-IDF: Quickly identified the most relevant keywords, helping us zero in on the core themes.
- Word Embeddings (Word2Vec): Showed us how words relate to each other, revealing hidden connections and synonyms.

How did vectorization empower you to find hidden connections?
- Turning words into numbers allowed us to use mathematical tools to spot patterns and similarities that would be hard to see with just our eyes.

What was the most surprising part of this adventure?
- I was amazed at how regular expressions uncovered a hidden code-like pattern – it felt like cracking a secret message!

How could you use these skills for other problems in the real world?
- Sentiment Analysis: Understanding the positive or negative "tone" of customer reviews or social media posts.
- Topic Modeling: Discovering the underlying themes in large collections of news articles or other texts.
- Better Search Engines: Creating search tools that find results based on the meaning of words, not just exact matches.

Thank you.

## 

##  Part 2 - Building an AI Agent 

In [41]:
# Import necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
import re


# Function to extract keywords using Word2Vec
def extract_keywords_word2vec(clues_list, keywords, n=5):
    # Tokenize the clues
    tokenized_clues = [clue.split() for clue in clues_list]

    # Create a Word2Vec model
    word2vec = Word2Vec(tokenized_clues, vector_size=100, window=5, min_count=1, sg=1)

    related_keywords = []
    for keyword_list in keywords:
        for keyword in keyword_list:
            try:
                # Get the most similar keywords for each input keyword
                related = word2vec.wv.most_similar(positive=[keyword], topn=n)
                related_keywords.extend([w[0] for w in related])
            except KeyError:
                # Skip keywords not in the vocabulary
                pass

    return related_keywords

# Example usage
text_data = [
    "Digital art encompasses a range of artistic work and practices that use digital technology as an essential part of the creative and/or presentation process.",
    "Space exploration is the investigation and discovery of outer space, including celestial bodies and phenomena.",
    "The human brain is a complex organ responsible for various cognitive functions and behaviors.",
]

clues = [
    "Digital art encompasses a range of artistic work and practices that use digital technology as an essential part of the creative and/or presentation process.",
    "Where polygons dance and shaders weave, my artistry helps the virtual world breathe.",
    "Tiny squares, a canvas wide, building images side by side.",
    "RGB spells my secret code, the building blocks where colors explode.",
]

# Clean text data and clues
cleaned_text_data = [clean_text(text) for text in text_data]
cleaned_clues = [clean_text(clue) for clue in clues]

# Extract keywords using TF-IDF
tfidf_keywords = extract_keywords_tfidf(cleaned_text_data, cleaned_clues)

# Extract keywords using Word2Vec
word2vec_keywords = extract_keywords_word2vec(cleaned_clues, tfidf_keywords)

# Detect patterns in clues
patterns = detect_patterns(cleaned_clues)

# Print the results
print("TF-IDF Keywords:")
for i, clue_keywords in enumerate(tfidf_keywords):
    print(f"Clue {i+1}: {', '.join(clue_keywords)}")

print("\nWord2Vec Keywords:")
for i, keyword in enumerate(word2vec_keywords):
    print(f"Keyword {i+1}: {keyword}")

print("\nDetected Patterns:")
for pattern in patterns:
    print(pattern)


TF-IDF Keywords:
Clue 1: andor, artistic, art
Clue 2: artistic, art, andor
Clue 3: artistic, art, andor
Clue 4: artistic, art, andor

Word2Vec Keywords:
Keyword 1: squares
Keyword 2: spells
Keyword 3: and
Keyword 4: process
Keyword 5: essential
Keyword 6: the
Keyword 7: artistry
Keyword 8: helps
Keyword 9: shaders
Keyword 10: canvas
Keyword 11: as
Keyword 12: helps
Keyword 13: my
Keyword 14: wide
Keyword 15: range
Keyword 16: the
Keyword 17: artistry
Keyword 18: helps
Keyword 19: shaders
Keyword 20: canvas
Keyword 21: as
Keyword 22: helps
Keyword 23: my
Keyword 24: wide
Keyword 25: range
Keyword 26: squares
Keyword 27: spells
Keyword 28: and
Keyword 29: process
Keyword 30: essential
Keyword 31: the
Keyword 32: artistry
Keyword 33: helps
Keyword 34: shaders
Keyword 35: canvas
Keyword 36: as
Keyword 37: helps
Keyword 38: my
Keyword 39: wide
Keyword 40: range
Keyword 41: squares
Keyword 42: spells
Keyword 43: and
Keyword 44: process
Keyword 45: essential
Keyword 46: the
Keyword 47: artist

### using a Trans: Take 2
#### gemime -slow, GPT - trumps 

In [56]:
# Import necessary libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
import re

# Function to clean text
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text.lower())  # Remove punctuation and convert to lowercase
    return text

## Function to detect patterns in clues using regular expressions
def detect_patterns(clues_list):
    patterns = []
    for clue in clues_list:
        # Improved patterns with some examples
        chemical_pattern = r"[A-Z][a-z]*[a-z0-9]*|[A-Z][0-9][a-z]*"  # Example: Serotonin, Dopamine
        science_term_pattern = r"\b(neurotransmitter|microbiome|receptor|enzyme)\b"  # Example list
        space_term_pattern = r"\b(rocket|satellite|astronaut|gravity|planet)\b"  # Example list
        celestial_pattern = r"\b(star|galaxy|black hole|constellation)\b"  # Example list

        chemicals = re.findall(chemical_pattern, clue, flags=re.IGNORECASE)
        science_terms = re.findall(science_term_pattern, clue, flags=re.IGNORECASE)
        space_terms = re.findall(space_term_pattern, clue, flags=re.IGNORECASE)
        celestial_terms = re.findall(celestial_pattern, clue, flags=re.IGNORECASE)

        if chemicals or science_terms or space_terms or celestial_terms:
            patterns.append({
                "clue": clue,
                "chemicals": chemicals,
                "science_terms": science_terms,
                "space_terms": space_terms,
                "celestial_terms": celestial_terms,
            })

    return patterns


# Function to extract keywords using TF-IDF
def extract_keywords_tfidf(text_data, clues):
    cleaned_text_data = [clean_text(text) for text in text_data]
    cleaned_clues = [clean_text(clue) for clue in clues]

    vectorizer = TfidfVectorizer(max_features=1000)  # Adjust max_features as needed
    tfidf_matrix = vectorizer.fit_transform(cleaned_text_data)
    clue_vectors = vectorizer.transform(cleaned_clues)

    tfidf_keywords = []
    for clue_vector, clue in zip(clue_vectors, cleaned_clues):
        # Get top n keywords with highest TF-IDF scores for each clue
        top_n_indices = clue_vector.indices[:5]  # Adjust number of keywords (n) as needed
        top_keywords = [vectorizer.get_feature_names_out()[i] for i in top_n_indices]
        tfidf_keywords.append(top_keywords)

    return tfidf_keywords


# Function to extract keywords using Word2Vec
def extract_keywords_word2vec(clues_list, keywords, n=5):
    # Tokenize the clues
    tokenized_clues = [clue.split() for clue in clues_list]

    # Create a Word2Vec model
    try:
        word2vec = Word2Vec(tokenized_clues, vector_size=100, window=5, min_count=1, sg=1)
    except:
        print("Error creating Word2Vec model. Skipping Word2Vec keyword extraction.")
        return []

    related_keywords = []
    for keyword_list in keywords:
        for keyword in keyword_list:
            try:
                # Get the most similar keywords for each input keyword
                related = word2vec.wv.most_similar(positive=[keyword], topn=n)
                related_keywords.extend([w[0] for w in related])
            except KeyError:
                # Skip keywords not in the vocabulary
                pass

    return related_keywords


# Example usage
text_data = [
    "Digital art encompasses a range of artistic work and practices that use digital technology as an essential part of the creative and/or presentation process.",
    "The digital art field has evolved significantly since the 1960s.",
    "Modern digital art can include both traditional forms of art, such as painting and sculpture, as well as new media forms like digital photography and interactive installations."
]

clues = [
    "Digital art encompasses a range of artistic work and practices that use digital technology as an essential part of the creative and/or presentation process.",
    "Digital art has evolved significantly since the 1960s.",
    "Modern digital art can include both traditional forms of art, such as painting and sculpture, as well as new media forms like digital photography and interactive installations."
]

# Extract keywords using TF-IDF
tfidf_keywords = extract_keywords_tfidf(text_data, clues)
print("TF-IDF Keywords:", tfidf_keywords)

# Extract keywords using Word2Vec
word2vec_keywords = extract_keywords_word2vec(clues, tfidf_keywords)
print("Word2Vec Keywords:", word2vec_keywords)

# Detect patterns in clues
patterns = detect_patterns(clues)
print("Patterns in Clues:", patterns)


TF-IDF Keywords: [['work', 'use', 'the', 'that', 'technology'], ['the', 'since', 'significantly', 'has', 'evolved'], ['well', 'traditional', 'such', 'sculpture', 'photography']]
Word2Vec Keywords: ['like', 'art', 'process.', 'Modern', 'part', 'of', 'art,', 'installations.', 'essential', 'and', 'include', 'media', 'photography', 'art,', 'a', 'include', 'painting', 'of', 'creative', 'practices', 'and/or', 'sculpture,', 'and', 'traditional', 'Modern', 'include', 'media', 'photography', 'art,', 'a', 'art', 'interactive', 'media', 'photography', 'an', 'interactive', 'the', 'painting', '1960s.', 'range', '1960s.', 'painting', 'practices', 'as', 'traditional', 'practices', 'traditional', 'painting', 'range', 'as', 'encompasses', 'presentation', 'of', 'both', 'Digital', 'evolved', 'essential', 'as', 'both', 'part', 'painting', 'a', 'can', 'an', 'art', 'of', 'the', 'media', 'since', 'can']
Patterns in Clues: [{'clue': 'Digital art encompasses a range of artistic work and practices that use digi

### End 